**Проект: система персональных рекомендаций для музыкального стримингового сервиса**  

Описание  
Цель проекта — создание прототипа работающей рекомендательной системы для Яндекс Музыки. Система должна помогать пользователям находить интересную музыку, основываясь на их вкусах и предпочтениях.  
  
Задачи  
Использовать данные о взаимодействиях примерно 1,4 млн пользователей с 1 млн треков.  
  
Построить пайплайн для расчёта персональных рекомендаций с помощью изученных алгоритмов машинного обучения.  

Разработать сервис рекомендаций.  

Данные  
Данные представлены в трёх файлах:  

tracks.parquet — информация о треках (track_id, albums, artists, genres).  

catalog_names.parquet — имена артистов, названия альбомов, треков и жанров (id, type, name).  

interactions.parquet — данные о том, какие пользователи прослушали тот или иной трек (user_id, track_id, track_seq, started_at).  

Инструменты  
Для выполнения проекта будут использованы:  

виртуальное окружение Python;

Jupyter Lab;

Visual Studio Code;

персональный S3-бакет.

Результаты  
Результатом работы станет набор скриптов в репозитории, выполнение которых позволит рассчитать персональные рекомендации и запустить соответствующий сервис.  



# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [ ]:
import logging
from statistics import mode
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import boto3
import polars as pl
import tqdm
from tqdm import tqdm, trange
from scipy.sparse import csr_matrix, hstack
from implicit.als import AlternatingLeastSquares

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import boto3
import os
import polars as pl
from collections import defaultdict
from catboost import CatBoostClassifier, Pool
import scipy
import cupy as cp

c:\Users\Admin\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
tracks = pd.read_parquet("data/tracks.parquet")
catalog_names = pd.read_parquet("data/catalog_names.parquet")

In [4]:
clickstream = pd.read_parquet("data/interactions.parquet")

In [5]:

def rename_list(lst, dic):
    """
    Replace a list of ids with the values from a dictionary
    """
    for i, element in enumerate(lst):
        if element in dic.keys():
            lst[i] = dic[element][0]
        else:
            lst[i] = np.nan
    return lst

In [6]:
tracks_to_catalog_dict ={'albums':'album', 'artists':'artist','genres':'genre','track_id':'track'}
for col in ['albums', 'artists', 'genres']:
    print('Starting renaming: {}'.format(col))
    cat_col = tracks_to_catalog_dict[col]
    renamer = dict(zip(catalog_names[catalog_names['type']==cat_col]['id'], zip(catalog_names[catalog_names['type']==cat_col]['name'])))
    tracks[col]=tracks[col].apply(lambda x: rename_list(list(x), renamer))
    print('Finished renaming: {}'.format(col))
renamer = dict(zip(catalog_names[catalog_names['type']=='track']['id'], zip(catalog_names[catalog_names['type']=='track']['name'])))
tracks['track_name']=tracks['track_id'].map(renamer)
tracks

Starting renaming: albums
Finished renaming: albums
Starting renaming: artists
Finished renaming: artists
Starting renaming: genres
Finished renaming: genres


,track_id,albums,artists,genres,track_name
0,26,"[Taller Children, Taller Children]",[Elizabeth & the Catapult],"[pop, folk]","(Complimentary Me,)"
1,38,"[Taller Children, Taller Children]",[Elizabeth & the Catapult],"[pop, folk]","(Momma's Boy,)"
2,135,"[Wild Young Hearts, Wild Young Hearts, Wild Yo...",[Noisettes],[pop],"(Atticus,)"
3,136,"[Wild Young Hearts, Wild Young Hearts, Wild Yo...",[Noisettes],[pop],"(24 Hours,)"
4,138,"[Wild Young Hearts, Wild Young Hearts, Don't U...",[Noisettes],[pop],"(Don't Upset The Rhythm (Go Baby Go),)"
...,...,...,...,...,...
999995,101478482,[На лицо],[FLESH],"[rusrap, rap]","(На лицо,)"
999996,101490148,[Без капли мысли],[Даня Милохин],"[pop, ruspop]","(Без капли мысли,)"
999997,101493057,[SKITTLES],[WhyBaby?],"[foreignrap, rap]","(SKITTLES,)"
999998,101495927,[Москва],[Yanix],"[rusrap, rap]","(Москва,)"


In [7]:
items = tracks.copy()
del tracks

# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [8]:
clickstream.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB


In [9]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   track_id    1000000 non-null  int64 
 1   albums      1000000 non-null  object
 2   artists     1000000 non-null  object
 3   genres      1000000 non-null  object
 4   track_name  1000000 non-null  object
dtypes: int64(1), object(4)
memory usage: 38.1+ MB


Пример данных по треку

In [10]:
items.sample(5)

,track_id,albums,artists,genres,track_name
439785,30593369,[100 рождественских хитов - Лучшие рождественс...,[Mario Lanza],[relax],"(Ave Maria,)"
968960,90132856,"[Live Performance 2020, Pt. 1]",[Ninety One],[pop],"(Why'M,)"
877886,72171659,[Девочка со вписки],[badCurt],"[rusrap, rap]","(Девочка со вписки,)"
245687,14729183,[Solarstone presents Pure Trance 2],"[Ferry Corsten, Betsie Larkin]",[],"(Made Of Love (Acapella),)"
461623,32189262,[Lights Out],[Virginia To Vegas],[dance],"(Lights Out,)"


Пример данных по действиям в приложении

In [11]:
clickstream.sample(5)

,user_id,track_id,track_seq,started_at
86,1115240,12694711,87,2022-09-25
8,893308,697295,9,2022-04-25
5454,882231,68075023,5455,2022-10-23
258,802207,71477779,259,2022-11-01
81,89609,328687,82,2022-02-08


In [12]:
items.isna().sum()

track_id      0
albums        0
artists       0
genres        0
track_name    0
dtype: int64

In [13]:
np.isnan(items['artists'].any()).sum()

np.int64(0)

In [14]:
# Подсчитываем количество строк, в которых в списках содержится np.nan
for col in ['albums', 'artists', 'genres','track_name']:
    print(col, items[col].apply(lambda x: None in x).sum())

albums 0
artists 0
genres 0
track_name 0


In [15]:
items[items['genres'].apply(lambda x: None in x)]

,track_id,albums,artists,genres,track_name


In [16]:
list1 = clickstream['track_id'].unique().tolist()
list2 = items['track_id'].unique().tolist()
array1 = np.array(list1)
array2 = np.array(list2)

count = np.sum(np.isin(array1, array2, invert=True))

print(count)

0


# Выводы

Приведём выводы по первому знакомству с данными:
- данных много, и явных проблем с ними не наблюдается
- был собран набор данных items, в которую из каталога перетащили информацию по трекам, это было сделано во-первых,   
для лучшего восприятия человеком, во-вторых, векторизация текстовых данных даст нам более полезные признаки, чем   
простое порядковое кодирование;  
- несколько смущает то, что в каждой ячейке items получился список, но пока это не создаёт проблем, посмотрим на этапе преобработки,  
может быть и поменяем типы;
- пропусков в данных практически нет, обнаружено что некоторые треки (менее 5%) отнесены к жанру None, но такк как это жанр не единственный, то  
сильно это картину не испортит;
- самое приятное, что в истории пользователей нет треков, которых не было бы в каталоге

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [17]:
print("Всего прослушано уникальных терков: ", clickstream["track_id"].nunique())

Всего прослушано уникальных терков:  1000000


Наиболее популярные треки

In [18]:
top10_tracks=clickstream["track_id"].value_counts().nlargest(10).index.tolist()
items[items['track_id'].isin(top10_tracks)]

,track_id,albums,artists,genres,track_name
9098,53404,"[Nevermind, Nirvana, Nevermind, Nevermind, Nev...",[Nirvana],"[alternative, rock, allrock]","(Smells Like Teen Spirit,)"
26665,178529,"[Meteora, Meteora, Meteora, Meteora, 00s Rock ...",[Linkin Park],"[numetal, metal]","(Numb,)"
90461,795836,"[Ten Summoner's Tales, 25 Years, The Best Of 2...",[Sting],"[pop, rock, allrock]","(Shape Of My Heart,)"
368072,24692821,"[Way down We Go, Summer Music 2016, A/B, DFM D...",[KALEO],[indie],"(Way Down We Go,)"
475289,32947997,"[Shape of You, ÷, ÷, Summer Vibes, Pop]",[Ed Sheeran],[pop],"(Shape of You,)"
483876,33311009,"[Shape Of Pop, NOW That's What I Call Music, E...",[Imagine Dragons],"[rock, allrock]","(Believer,)"
512157,35505245,"[I Got Love, I Got Love]","[Miyagi & Эндшпиль, Рем Дигга]","[rusrap, rap]","(I Got Love,)"
647237,45499814,"[Life, Life, Made in Russia, Fresh Dance, Life...",[Zivert],"[pop, ruspop]","(Life,)"
696106,51241318,"[In the End, Christian TikTok, Trending Now 20...","[Tommee Profitt, Fleurie, Mellen Gi]",[rnb],"(In The End,)"
829320,65851540,[Юность],[Dabro],"[pop, ruspop]","(Юность,)"


Наиболее популярные жанры

In [19]:
di = dict(zip(items['track_id'], items['genres']))
out = pd.DataFrame(columns=['genres','count'])
for i in range(5):
    sub = clickstream.sample(10000)
    sub['genres'] = sub['track_id'].map(di)
    sub['genres'] = sub['genres'].apply(lambda x: str(x))
    out1 = pd.DataFrame(dict(sub['genres'].value_counts().nlargest(10)).items(), columns=['genres','count'])
    out = pd.merge(out, out1, how='outer', on='genres', suffixes=(None, str(i)))

In [20]:
out

,genres,count,count0,count1,count2,count3,count4
0,['alternative'],NaN,337.0,358.0,353.0,317.0,323.0
1,['dance'],NaN,665.0,665.0,665.0,650.0,650.0
2,['electronics'],NaN,468.0,501.0,501.0,463.0,501.0
3,"['foreignrap', 'rap']",NaN,533.0,536.0,536.0,526.0,528.0
4,['indie'],NaN,263.0,256.0,253.0,NaN,247.0
5,"['metal', nan]",NaN,NaN,NaN,NaN,232.0,NaN
6,"['pop', 'ruspop']",NaN,1212.0,1227.0,1185.0,1236.0,1165.0
7,['pop'],NaN,1079.0,994.0,1049.0,1080.0,1017.0
8,"['rock', 'allrock']",NaN,416.0,424.0,382.0,407.0,396.0
9,"['rusrap', 'rap']",NaN,1087.0,1150.0,1187.0,1108.0,1118.0


In [21]:
out['total'] = out[['count0',	'count1',	'count2',	'count3',	'count4']].sum(axis=1)
out.sort_values(by='total',ascending=False).head(10)

,genres,count,count0,count1,count2,count3,count4,total
6,"['pop', 'ruspop']",NaN,1212.0,1227.0,1185.0,1236.0,1165.0,6025.0
9,"['rusrap', 'rap']",NaN,1087.0,1150.0,1187.0,1108.0,1118.0,5650.0
7,['pop'],NaN,1079.0,994.0,1049.0,1080.0,1017.0,5219.0
1,['dance'],NaN,665.0,665.0,665.0,650.0,650.0,3295.0
10,"['rusrock', 'allrock']",NaN,581.0,575.0,564.0,621.0,578.0,2919.0
3,"['foreignrap', 'rap']",NaN,533.0,536.0,536.0,526.0,528.0,2659.0
2,['electronics'],NaN,468.0,501.0,501.0,463.0,501.0,2434.0
8,"['rock', 'allrock']",NaN,416.0,424.0,382.0,407.0,396.0,2025.0
0,['alternative'],NaN,337.0,358.0,353.0,317.0,323.0,1688.0
4,['indie'],NaN,263.0,256.0,253.0,NaN,247.0,1019.0


Треки, которые никто не прослушал

In [22]:
list1 = clickstream['track_id'].unique().tolist()
list2 = items['track_id'].unique().tolist()
array1 = np.array(list1)
array2 = np.array(list2)

count = np.sum(np.isin(array2, array1, invert=True))

print(count)

0


# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [23]:
for col in ['albums', 'artists', 'genres','track_name']:
    items[col]=items[col].apply(lambda x: str(x))

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [24]:
items.to_parquet("data/items.parquet")
clickstream.to_parquet("data/events.parquet")

*Чтобы следующи код отработал хорошо, понадобится создать файл 'config.txt' в корневой папке проекта с содержимым:  
```
# Yandex Cloud S3 configuration  
AWS_ACCESS_KEY_ID=YC______________k  
AWS_SECRET_ACCESS_KEY=YC__________________S  
S3_SERVICE_NAME=s3  
S3_ENDPOINT_URL=https://storage.yandexcloud.net  
```


In [25]:

class Config:
    _config_loaded = False

    @classmethod
    def load_config(cls, filename='config.txt'):
        """Загружает конфигурацию из файла"""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Config file {filename} not found")

        with open(filename, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    setattr(cls, key.strip(), value.strip())
        
        cls._config_loaded = True

    @classmethod
    def get(cls, key):
        if not cls._config_loaded:
            cls.load_config()
        return getattr(cls, key, None)

def get_session():
    # Загружаем конфиг при первом вызове
    if not Config._config_loaded:
        Config.load_config()
    
    return boto3.session.Session().client(
        service_name=Config.get('S3_SERVICE_NAME'),
        endpoint_url=Config.get('S3_ENDPOINT_URL'),
        aws_access_key_id=Config.get('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=Config.get('AWS_SECRET_ACCESS_KEY')
    )

def upload_files_to_s3(file_paths: list, s3_bucket: str, s3_prefix: str = 'recsys/data/'):
    """Загружает файлы в S3-бакет по указанному пути"""
    s3 = get_session()
    
    for local_path in file_paths:
        if not os.path.exists(local_path):
            raise FileNotFoundError(f"Файл {local_path} не найден")
            
        s3_key = os.path.join(s3_prefix, os.path.basename(local_path))
        
        with open(local_path, 'rb') as f:
            s3.upload_fileobj(
                Fileobj=f,
                Bucket=s3_bucket,
                Key=s3_key
            )
        print(f"Файл {local_path} успешно загружен в s3://{s3_bucket}/{s3_key}")


bucket_name = 's3-student-mle-20250130-833968fcc1'

files_to_upload = [
        'data\items.parquet',
        'data\events.parquet',
    ]

# Загрузка файлов
upload_files_to_s3(
        file_paths=files_to_upload,
        s3_bucket=bucket_name
    )


<>:59: SyntaxWarning: invalid escape sequence '\i'
<>:60: SyntaxWarning: invalid escape sequence '\e'
<>:59: SyntaxWarning: invalid escape sequence '\i'
<>:60: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Admin\AppData\Local\Temp\ipykernel_3388\882964279.py:59: SyntaxWarning: invalid escape sequence '\i'
  'data\items.parquet',
C:\Users\Admin\AppData\Local\Temp\ipykernel_3388\882964279.py:60: SyntaxWarning: invalid escape sequence '\e'
  'data\events.parquet',


Файл data\items.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/items.parquet
Файл data\events.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/events.parquet


# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

In [26]:
del items, clickstream
del catalog_names
del list1, list2, array1, array2
del top10_tracks, di, out, out1
del count
del sub

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

In [2]:
items = pd.read_parquet("data/items.parquet")
events = pd.read_parquet("data/events.parquet")

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

In [3]:
# В качестве точки разбиения используйте 16 декабря 2022 года
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16").date()
events["started_at"] = pd.to_datetime(events["started_at"]).dt.date
train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = list(set(users_train.values).intersection(users_test.values))
print(len(users_train), len(users_test), len(common_users))

1342566 783525 752870


In [29]:
events["started_at"].min(), events["started_at"].max()

(datetime.date(2022, 1, 1), datetime.date(2022, 12, 31))

# Топ популярных

Рассчитаем рекомендации как топ популярных.

In [30]:
common_train = events_train[events_train['user_id'].isin(common_users)]
arr = np.array(common_train['track_id'])
unique, counts = np.unique(arr, return_counts=True)

In [31]:
counts = [int(x) for x in counts]
a = pd.DataFrame(columns=['track_id', 'count'])
a = a.assign(track_id=unique, count=counts)
a = a.sort_values(by='count', ascending=False)
most_popular_tracks = a.head(1000)
most_popular_tracks_with_names = most_popular_tracks.merge(items[['track_id','track_name']], on='track_id', how='left')
most_popular_tracks_with_names = most_popular_tracks_with_names.merge(items[['track_id','genres']], on='track_id', how='left')
most_popular_tracks_with_names


,track_id,count,track_name,genres
0,53404,65607,"('Smells Like Teen Spirit',)","['alternative', 'rock', 'allrock']"
1,178529,60183,"('Numb',)","['numetal', 'metal']"
2,33311009,57799,"('Believer',)","['rock', 'allrock']"
3,35505245,54426,"('I Got Love',)","['rusrap', 'rap']"
4,24692821,49963,"('Way Down We Go',)",['indie']
...,...,...,...,...
995,37989843,10619,"('Broken People',)","['foreignrap', 'rap']"
996,23616681,10609,"('Мелом',)","['pop', 'ruspop']"
997,61139585,10598,"('Река',)","['local-indie', 'indie']"
998,24663744,10593,"('Undress Rehearsal',)",['pop']


Теперь посчитаем рекомендации на основе наиболее популярных терков для всего тестового набора.

In [32]:
# Определим любимый жанр для каждого пользователя, по которому у нас есть история прослушиваний
items = pl.from_pandas(items)
events_test = pl.from_pandas(events_test)
#Фильтруем events_test для получения только тех записей, где user_id присутствует в common_users
user_events = events_test.filter(pl.col("user_id").is_in(common_users))

# Соединяем отфильтрованный DataFrame с items по track_id
user_events_items = user_events.join(items.select(["track_id", "genres"]), on="track_id", how="inner")

# Группируем данные по user_id и genres, и считаем количество прослушиваний каждого жанра для каждого пользователя
user_genre_counts = user_events_items.group_by(["user_id", "genres"]).len().sort(["user_id",'len'], descending=True)


# Находим жанр с максимальным количеством прослушиваний для каждого пользователя
most_frequent_genre = user_genre_counts.group_by('user_id').head(1)
#])

# Отображаем результат
most_frequent_genre_dict = dict(zip(most_frequent_genre['user_id'], most_frequent_genre['genres']))
most_frequent_genre_dict


{1374582: "['pop', 'ruspop']",
 1374581: "['pop', 'ruspop']",
 1374578: "['dance']",
 1374575: "['pop', 'ruspop']",
 1374573: "['pop', 'ruspop']",
 1374571: "['electronics', 'house']",
 1374568: "['rusrock', 'allrock']",
 1374567: "['rock', 'allrock']",
 1374566: "['pop', 'ruspop']",
 1374563: "['pop', 'ruspop']",
 1374562: "['rusestrada', 'estrada']",
 1374561: "['rusrap', 'rap']",
 1374558: "['pop', 'ruspop']",
 1374557: "['pop']",
 1374556: "['dance']",
 1374555: "['pop', 'ruspop']",
 1374552: "['pop', 'ruspop']",
 1374551: "['pop']",
 1374549: "['metal', 'industrial']",
 1374546: "['foreignrap', 'rap']",
 1374545: "['podcasts', 'education', 'health', 'society']",
 1374541: "['pop']",
 1374539: "['pop', 'ruspop']",
 1374536: "['pop', 'ruspop']",
 1374535: "['pop']",
 1374532: "['pop', 'ruspop']",
 1374530: "['foreignrap', 'rap']",
 1374529: "['rusrap', 'rap']",
 1374527: "['pop']",
 1374526: "['rusrap', 'rap']",
 1374524: "['pop', 'ruspop']",
 1374521: "['pop']",
 1374520: "['punk']

In [34]:
track_ids = most_popular_tracks_with_names['track_id'].to_numpy()
genres = most_popular_tracks_with_names['genres'].to_numpy()

# Создаем списки для хранения рекомендаций
user_ids_list = []
recommended_track_ids_list = []

# Проходим по каждому пользователю
for user in tqdm(users_test.to_list()):
    try:
        # Получаем любимый жанр пользователя
        g = most_frequent_genre_dict[user]
        
        # Фильтруем track_ids по любимому жанру пользователя
        filtered_track_ids = track_ids[genres == g]
        
        # Выбираем 10 случайных треков из отфильтрованных
        if len(filtered_track_ids) >= 10:
            ten_recs = np.random.choice(filtered_track_ids, size=10, replace=False)
        else:
            # Если не хватает треков, берем все доступные и добавляем случайные из оставшихся
            ten_recs = np.concatenate([
                filtered_track_ids,
                np.random.choice(track_ids[genres != g], size=10 - len(filtered_track_ids), replace=False)
            ])
        
        # Добавляем рекомендации в списки
        user_ids_list.extend([user] * 10)
        recommended_track_ids_list.extend(ten_recs.tolist())
    except KeyError:
        # Пользователям без любимого жанра рекомендуем самые популярные треки
        ten_recs = np.random.choice(track_ids, size=10, replace=False)
        user_ids_list.extend([user] * 10)
        recommended_track_ids_list.extend(ten_recs.tolist())

# Создаем итоговый DataFrame из списков рекомендаций
pop_recs = pl.DataFrame({
    "user_id": user_ids_list,
    "recommended_track_id": recommended_track_ids_list
})

# Отображаем результат
pop_recs

100%|██████████| 783525/783525 [00:27<00:00, 28371.43it/s]


user_id,recommended_track_id
i64,i64
3,57502030
3,74683925
3,42242123
3,61843244
3,67538121
…,…
1374582,36041535
1374582,41516796
1374582,65320306


In [35]:
pd.DataFrame(pop_recs, columns=['user_id', 'recommended_track_id']).to_parquet("data/top_popular.parquet")

In [36]:
upload_files_to_s3(
        file_paths=['data/top_popular.parquet'],
        s3_bucket=bucket_name
    )

Файл data/top_popular.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/top_popular.parquet


# Персональные

Рассчитаем персональные рекомендации.

In [37]:
# Создаем маппинг user_id и track_id в числовые индексы
users = list(common_train['user_id'].unique())
tracks = list(common_train['track_id'].unique())
    
user_map = {int(user): idx for idx, user in enumerate(users)}
track_map = {int(track): idx for idx, track in enumerate(tracks)}
common_train =pl.from_pandas(common_train)    
# Преобразуем user_id и track_id в числовые индексы
common_train = common_train.with_columns([
        pl.col("user_id").replace(user_map, default=-1).alias("user_idx"),
        pl.col("track_id").replace(track_map, default=-1).alias("track_idx")
    ])

C:\Users\Admin\AppData\Local\Temp\ipykernel_3388\43583893.py:10: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("user_id").replace(user_map, default=-1).alias("user_idx"),
C:\Users\Admin\AppData\Local\Temp\ipykernel_3388\43583893.py:11: DeprecationWarning: The `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
  pl.col("track_id").replace(track_map, default=-1).alias("track_idx")


In [38]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    np.ones(len(common_train)),
    (common_train['user_idx'].to_numpy(), common_train['track_idx'].to_numpy())),
    dtype=np.int8)

In [39]:

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train) 

c:\Users\Admin\miniconda3\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 20 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [13:25<00:00, 16.10s/it]


In [40]:
def get_recommendations_als(user_item_matrix, model, user_id, user_map, track_map, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    # Получаем закодированный user_id
    user_id_enc = user_map[user_id]
    
    # Создаем обратный словарь для track_map
    inv_track_map = {v: k for k, v in track_map.items()}
    
    # Получаем рекомендации от модели
    recommendations = model.recommend(
        user_id_enc, 
        user_item_matrix[user_id_enc], 
        filter_already_liked_items=not include_seen,
        N=n
    )
    
    # Разбираем рекомендации на track_idx и score
    track_indices = recommendations[0]
    scores = recommendations[1]
    
    # Преобразуем track_idx в track_id с помощью inv_track_map
    track_ids = np.array([inv_track_map.get(idx, -1) for idx in track_indices])
    
    # Создаем итоговый DataFrame с помощью Polars
    recommendations_df = pl.DataFrame({
        "track_id": track_ids,
        "score": scores
    })
    
    return recommendations_df

In [41]:
r = get_recommendations_als(user_item_matrix_train, als_model, 22, user_map, track_map, include_seen=True, n=5)
np.array(r, dtype=int)[:,0]

array([54798445, 49961817, 45499814, 62352387, 69459326])

In [42]:
del common_train

In [43]:
# получаем список всех возможных user_id (перекодированных) для которых можно посчитать персональные рекомендации на основании матрицы схожести
user_ids_encoded = [user_map[int(x)] for x in common_users]

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=True, N=100)

In [44]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})

In [45]:
# Разворачиваем (exploding) столбцы item_id_enc и score в отдельные строки для каждого элемента списка
# ignore_index=True позволяет сбросить индекс результирующего DataFrame, чтобы индексы были последовательными
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)
als_recommendations


,user_id_enc,item_id_enc,score
0,0,688,0.295663
1,0,345,0.27522
2,0,717,0.246964
3,0,682,0.241626
4,0,742,0.210551
...,...,...,...
75286995,752869,20,0.084045
75286996,752869,2550,0.084006
75286997,752869,7910,0.083638
75286998,752869,2473,0.083579


In [46]:
# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")
inv_track_map = {v: k for k, v in track_map.items()}
inv_user_map = {v: k for k, v in user_map.items()}
# получаем изначальные идентификаторы
als_recommendations["user_id"] =als_recommendations["user_id_enc"].map(inv_user_map)
als_recommendations["track_id"] = als_recommendations["item_id_enc"].map(inv_track_map)
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])
als_recommendations

,score,user_id,track_id
0,0.295663,3,45499814
1,0.275220,3,33311009
2,0.246964,3,54798445
3,0.241626,3,39946957
4,0.210551,3,57921154
...,...,...,...
75286995,0.084045,1374582,48591706
75286996,0.084006,1374582,47201922
75286997,0.083638,1374582,76030982
75286998,0.083579,1374582,33509648


In [47]:
common_users_list = [int(x) for x in common_users]

In [48]:
mask = (~events_test['user_id'].is_in(common_users_list))
filtered_events_test = events_test.filter(mask)

In [49]:
recommended_track_ids_list=[]
user_ids_list=[]
non_common_users = list(filtered_events_test['user_id'].unique())
for user in non_common_users:
    ten_recs = np.random.choice(track_ids, size=10, replace=False)
    user_ids_list.extend([user] * 10)
    recommended_track_ids_list.extend(ten_recs.tolist())
recommended_tracks_df = pd.DataFrame({'user_id': user_ids_list, 'track_id': recommended_track_ids_list})
recommended_tracks_df = pd.concat([recommended_tracks_df, als_recommendations[['user_id', 'track_id']]], axis=0)
recommended_tracks_df.rename(columns={'track_id': 'recommended_track_id'}, inplace=True)
recommended_tracks_df.to_parquet('data/personal_als.parquet')

In [50]:
als_recommendations.to_parquet('data/als_scores.parquet')

In [51]:
upload_files_to_s3(
        file_paths=['data/personal_als.parquet'],
        s3_bucket=bucket_name
    )

Файл data/personal_als.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/personal_als.parquet


# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

In [52]:
len(tracks)

990742

In [53]:
# Assuming als_model.similar_items can handle csr_matrix or converted NumPy arrays
# Convert your data to CuPy if needed
tracks_gpu = cp.array([track_map[track] for track in tracks])

# Function to get similar items in batches and concatenate results
def get_similar_tracks_batchwise(tracks_gpu, batch_size=10240):
    all_similar_tracks = []
    all_similarity_scores = []
    all_index = []
    
    for i in trange(0, len(tracks_gpu), batch_size):
        batch = tracks_gpu[i:i + batch_size].get()  # Convert CuPy array to NumPy array
        similar_tracks_batch, similarity_scores_batch = als_model.similar_items(batch, N=10)
        
        # Ensure the results are CuPy arrays
        all_similar_tracks.append(cp.array(similar_tracks_batch))
        all_similarity_scores.append(cp.array(similarity_scores_batch))
        all_index.append(cp.array([tracks[i]] * 10 * len(similarity_scores_batch)))
    
    # Concatenate all results
    all_similar_tracks = cp.concatenate(all_similar_tracks, axis=0)
    all_similarity_scores = cp.concatenate(all_similarity_scores, axis=0)
    all_index = cp.concatenate(all_index, axis=0)
    
    return all_similar_tracks, all_similarity_scores, all_index

all_similar_tracks, all_similarity_scores, all_index = get_similar_tracks_batchwise(tracks_gpu)

100%|██████████| 97/97 [38:43<00:00, 23.96s/it]


In [54]:
# Convert the keys and values of track_map to CuPy arrays
keys = cp.array(list(track_map.keys()), dtype=cp.int32)
values = cp.array(list(track_map.values()), dtype=cp.int32)

# Create a reverse mapping using CuPy
rev_track_map = cp.zeros_like(values, dtype=cp.int32)
rev_track_map[values] = keys

# Convert all_similar_tracks to a CuPy array
all_similar_tracks_gpu = cp.array(all_similar_tracks, dtype=cp.int32)

# Use CuPy to perform the mapping
all_similar_tracks_real_id_gpu = rev_track_map[all_similar_tracks_gpu]

# Convert the result back to a NumPy array if needed
all_similar_tracks_real_id = all_similar_tracks_real_id_gpu.get()

all_similar_tracks_real_id

array([[ 6006252, 20100132, 30380200, ..., 15451558,  2214166, 34391227],
       [21642261, 20266522, 34659731, ..., 32658772, 26699240, 23821144],
       [21642265, 27183263, 28383438, ..., 27184513, 27769199, 25657261],
       ...,
       [94380812, 88793332, 80811063, ..., 96028064, 80236773, 77494449],
       [94642241, 92794642, 75519047, ..., 36704947, 38571339, 74272255],
       [81754412, 63307319, 79478520, ..., 73767216, 97124963, 66803804]],
      shape=(990742, 10), dtype=int32)

In [55]:
# Создаем двумерный массив с помощью CuPy
num_indices = all_index.shape[0]
num_tracks_per_index = all_similar_tracks_real_id.shape[1]

#result = cp.zeros((num_indices * num_tracks_per_index, 3), dtype=cp.float32)

# Заполняем двумерный массив векторизованными операциями
index_column = cp.array(all_index)
track_column = cp.array(all_similar_tracks_real_id.flatten())
score_column = cp.array(all_similarity_scores.flatten())



# Преобразуем результат обратно в NumPy массив, если это необходимо
result = cp.column_stack((index_column, track_column, score_column))
result_np = result.get()
print(result_np)

[[6.00625200e+06 6.00625200e+06 9.99999940e-01]
 [6.00625200e+06 2.01001320e+07 9.53783274e-01]
 [6.00625200e+06 3.03802000e+07 9.15689409e-01]
 ...
 [3.69195900e+06 7.37672160e+07 6.42166615e-01]
 [3.69195900e+06 9.71249630e+07 6.39615357e-01]
 [3.69195900e+06 6.68038040e+07 6.39598370e-01]]


In [56]:
similarity_df = pd.DataFrame(result_np, columns=['user_id', 'item_id', 'score'])
similarity_df['user_id'] = similarity_df['user_id'].astype(int)
similarity_df['item_id'] = similarity_df['item_id'].astype(int)
similarity_df

,user_id,item_id,score
0,6006252,6006252,1.000000
1,6006252,20100132,0.953783
2,6006252,30380200,0.915689
3,6006252,15451554,0.911454
4,6006252,582507,0.907848
...,...,...,...
9907415,3691959,67132145,0.647430
9907416,3691959,89994974,0.643437
9907417,3691959,73767216,0.642167
9907418,3691959,97124963,0.639615


In [57]:
similarity_df.to_parquet('data/similar.parquet')

In [58]:
upload_files_to_s3(
        file_paths=['data/similar.parquet'],
        s3_bucket=bucket_name
    )

Файл data/similar.parquet успешно загружен в s3://s3-student-mle-20250130-833968fcc1/recsys/data/similar.parquet


# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

In [59]:
#Первый признак - оценка по ALS модели
als_scores = pd.read_parquet("data/als_scores.parquet")
als_scores

,score,user_id,track_id
0,0.295663,3,45499814
1,0.275220,3,33311009
2,0.246964,3,54798445
3,0.241626,3,39946957
4,0.210551,3,57921154
...,...,...,...
75286995,0.084045,1374582,48591706
75286996,0.084006,1374582,47201922
75286997,0.083638,1374582,76030982
75286998,0.083579,1374582,33509648


In [60]:
tracks = pd.read_parquet("data/tracks.parquet")
tracks

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]
...,...,...,...,...
999995,101478482,[21399811],[5540395],"[3, 75]"
999996,101490148,[21403052],[9078726],"[11, 20]"
999997,101493057,[21403883],[11865715],"[44, 75]"
999998,101495927,[21404975],[4462686],"[3, 75]"


In [61]:
def add_recommendation_features(events, items):
    """
    Добавляет 2 новых признака для рекомендательной системы:
    1. user_genre_affinity - нормированная частота прослушиваний жанра
    2. user_artist_ratio - доля прослушиваний исполнителя
    
    Параметры:
    events - DataFrame с колонками ['user_id', 'track_id']
    items - DataFrame с колонками ['track_id', 'artist', 'genre']
    
    Возвращает:
    Модифицированный DataFrame events с добавленными признаками
    """
    
    # 1. Создание быстрых lookup-таблиц
    genre_lookup = items.set_index('track_id')['genre'].to_dict()
    artist_lookup = items.set_index('track_id')['artist'].to_dict()
    
    # 2. Инициализация структур для хранения статистик
    user_stats = {
        'genre': defaultdict(lambda: defaultdict(int)),
        'artist': defaultdict(lambda: defaultdict(int)),
        'total': defaultdict(int)
    }
    
    # 3. Однопроходный расчет статистик
    for _, row in events.iterrows():
        user_id = row['user_id']
        track_id = row['track_id']
        
        # Получаем жанр и исполнителя
        genre = genre_lookup.get(track_id)
        artist = artist_lookup.get(track_id)
        
        # Обновляем счетчики
        if genre:
            user_stats['genre'][user_id][genre] += 1
        if artist:
            user_stats['artist'][user_id][artist] += 1
        
        user_stats['total'][user_id] += 1
    
    # 4. Функции для расчета признаков
    def calc_genre_affinity(row):
        user_id = row['user_id']
        track_id = row['track_id']
        genre = genre_lookup.get(track_id)
        
        if not genre or user_stats['total'][user_id] == 0:
            return 0.0
        return user_stats['genre'][user_id][genre] / user_stats['total'][user_id]
    
    def calc_artist_ratio(row):
        user_id = row['user_id']
        track_id = row['track_id']
        artist = artist_lookup.get(track_id)
        
        if not artist or user_stats['total'][user_id] == 0:
            return 0.0
        return user_stats['artist'][user_id][artist] / user_stats['total'][user_id]
    
    # 5. Добавление признаков к данным
    events = events.copy()
    events['user_genre_affinity'] = events.apply(calc_genre_affinity, axis=1)
    events['user_artist_ratio'] = events.apply(calc_artist_ratio, axis=1)
    
    return events

In [66]:
events1 = events_train[['user_id','track_id']]
items1 = items[['track_id','artists','genres']]
# Renaming columns in a Polars DataFrame
items1 = items1.with_columns([
    pl.col('artists').alias('artist'),
    pl.col('genres').alias('genre')
])

In [67]:
try:
    enhanced_events = pd.read_parquet('data/enchanced_events.parquet')
except:
    enhanced_events = add_recommendation_features(events1, items1)
enhanced_events

,user_id,track_id,user_genre_affinity,user_artist_ratio
0,0,99262,0.076923,0.038462
1,0,589498,0.192308,0.153846
2,0,590262,0.192308,0.153846
3,0,590303,0.192308,0.153846
4,0,590692,0.192308,0.153846
...,...,...,...,...
208731247,1374582,76512143,0.285024,0.004831
208731248,1374582,76820953,0.285024,0.004831
208731249,1374582,77549370,0.125604,0.004831
208731250,1374582,77590298,0.086957,0.004831


In [69]:
enhanced_events.to_parquet('data/enchanced_events.parquet')

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

In [4]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2022-12-22").date()

# конвертируем столбец started_at к типу date
events_test["started_at"] = pd.to_datetime(events_test["started_at"])
events_test["started_at"] = events_test["started_at"].dt.date

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()

C:\Users\Admin\AppData\Local\Temp\ipykernel_13184\3553269422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["started_at"] = pd.to_datetime(events_test["started_at"])
C:\Users\Admin\AppData\Local\Temp\ipykernel_13184\3553269422.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["started_at"] = events_test["started_at"].dt.date


In [5]:
events_test_2 = events_test[~split_date_for_labels_idx].copy() 

In [8]:
events_test_2.to_parquet('data/events_test_2.parquet')

In [8]:
enhanced_events = pl.read_parquet('data\enchanced_events.parquet')

<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
C:\Users\Admin\AppData\Local\Temp\ipykernel_812\2955894375.py:1: SyntaxWarning: invalid escape sequence '\e'
  enhanced_events = pl.read_parquet('data\enchanced_events.parquet')


In [6]:
als_recommendations = pl.read_parquet(r'data\als_scores.parquet')

In [9]:
candidates = (
    als_recommendations.select(["user_id", "track_id", "score"])
    .with_columns(pl.col("score").alias("als_score"))
    .join(
        enhanced_events.select(["user_id", "track_id", "user_genre_affinity", "user_artist_ratio"]),
        on=["user_id", "track_id"],
        how="outer"
    )
)

candidates

C:\Users\Admin\AppData\Local\Temp\ipykernel_812\1842607707.py:4: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(


user_id,track_id,score,als_score,user_id_right,track_id_right,user_genre_affinity,user_artist_ratio
i64,i64,f64,f64,i32,i32,f64,f64
null,null,null,null,0,99262,0.076923,0.038462
null,null,null,null,0,589498,0.192308,0.153846
null,null,null,null,0,590262,0.192308,0.153846
null,null,null,null,0,590303,0.192308,0.153846
null,null,null,null,0,590692,0.192308,0.153846
…,…,…,…,…,…,…,…
595341,60292250,0.011393,0.011393,null,null,null,null
175852,24522533,0.004729,0.004729,null,null,null,null
831745,41516796,0.004023,0.004023,null,null,null,null


In [10]:
# Заполняем пропуски в столбце 'user_id' соответствующими значениями из 'user_id_right'
candidates = candidates.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates = candidates.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))

candidates = candidates.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio'])
candidates.write_parquet('data/candidates.parquet')

In [11]:
events_labels = pl.from_pandas(events_labels)
# Add the target column to events_labels with a value of 1
events_labels = events_labels.with_columns(pl.lit(1).alias("target"))

# Perform an outer join on user_id and track_id
candidates = (
    events_labels.select(["user_id", "track_id", "target"])
    .join(candidates, 
    on=["user_id", "track_id"], 
    how="outer")
)

# Fill null values in the target column with 0 and convert to integer
candidates = candidates.with_columns(
    pl.col("target").fill_null(0).cast(pl.Int32)
)

# Sample 50 rows from the candidates DataFrame
print(candidates.sample(50))

C:\Users\Admin\AppData\Local\Temp\ipykernel_812\3704159602.py:8: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(candidates,


shape: (50, 8)
┌─────────┬──────────┬────────┬──────────────┬─────────────┬───────────┬─────────────┬─────────────┐
│ user_id ┆ track_id ┆ target ┆ user_id_righ ┆ track_id_ri ┆ als_score ┆ user_genre_ ┆ user_artist │
│ ---     ┆ ---      ┆ ---    ┆ t            ┆ ght         ┆ ---       ┆ affinity    ┆ _ratio      │
│ i32     ┆ i32      ┆ i32    ┆ ---          ┆ ---         ┆ f64       ┆ ---         ┆ ---         │
│         ┆          ┆        ┆ i64          ┆ i64         ┆           ┆ f64         ┆ f64         │
╞═════════╪══════════╪════════╪══════════════╪═════════════╪═══════════╪═════════════╪═════════════╡
│ null    ┆ null     ┆ 0      ┆ 1123908      ┆ 81848875    ┆ null      ┆ 0.25        ┆ 0.013158    │
│ null    ┆ null     ┆ 0      ┆ 850152       ┆ 37182648    ┆ null      ┆ 0.325459    ┆ 0.057743    │
│ null    ┆ null     ┆ 0      ┆ 1273156      ┆ 835443      ┆ null      ┆ 0.034483    ┆ 0.001301    │
│ null    ┆ null     ┆ 0      ┆ 401721       ┆ 44308193    ┆ null      ┆ 0.2

In [12]:

candidates = candidates.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates = candidates.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))
candidates = candidates.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio','target'])
candidates.write_parquet('data/candidates_target.parquet')

In [11]:
candidates = pl.read_parquet('data/candidates_target.parquet')

In [14]:
# Выбираем кандидатов, у которых хотя бы в одной строке 'target' равно 1
candidates_with_target_1 = candidates.filter(pl.col("target") == 1)
user_id_list = list(candidates_with_target_1.select("user_id").unique())[0]
selected_candidates = candidates.filter(pl.col("user_id").is_in(user_id_list))
# Выбираем ещё 4 примера, где 'target' равно 0 для каждого 'user_id'
candidates_with_target_0 = selected_candidates.filter(pl.col("target") == 0).group_by("user_id").head(4)

# Объединяем выбранные кандидаты
filtered_candidates = pl.concat([candidates_with_target_1, candidates_with_target_0])

print(filtered_candidates)

shape: (8_946_262, 6)
┌─────────┬──────────┬───────────┬─────────────────────┬───────────────────┬────────┐
│ user_id ┆ track_id ┆ als_score ┆ user_genre_affinity ┆ user_artist_ratio ┆ target │
│ ---     ┆ ---      ┆ ---       ┆ ---                 ┆ ---               ┆ ---    │
│ i64     ┆ i64      ┆ f64       ┆ f64                 ┆ f64               ┆ i32    │
╞═════════╪══════════╪═══════════╪═════════════════════╪═══════════════════╪════════╡
│ 47521   ┆ 50806996 ┆ 0.013327  ┆ null                ┆ null              ┆ 1      │
│ 139606  ┆ 25972219 ┆ 0.00709   ┆ null                ┆ null              ┆ 1      │
│ 296886  ┆ 631630   ┆ 0.044804  ┆ null                ┆ null              ┆ 1      │
│ 194592  ┆ 18227581 ┆ 0.025229  ┆ null                ┆ null              ┆ 1      │
│ 692637  ┆ 47627256 ┆ 0.07313   ┆ null                ┆ null              ┆ 1      │
│ …       ┆ …        ┆ …         ┆ …                   ┆ …                 ┆ …      │
│ 628386  ┆ 18046005 ┆ null     

In [15]:
# задаём имена колонок признаков и таргета
features = ['als_score', 'user_genre_affinity', 'user_artist_ratio']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=np.array(filtered_candidates[features]), 
    label=np.array(filtered_candidates[target])
)

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=10,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data) 

0:	learn: 0.5980682	total: 350ms	remaining: 34.6s
10:	learn: 0.1787272	total: 2.73s	remaining: 22.1s
20:	learn: 0.0631435	total: 5.04s	remaining: 18.9s
30:	learn: 0.0238753	total: 7.21s	remaining: 16s
40:	learn: 0.0097898	total: 9.36s	remaining: 13.5s
50:	learn: 0.0047402	total: 11.5s	remaining: 11.1s
60:	learn: 0.0028586	total: 13.7s	remaining: 8.74s
70:	learn: 0.0021379	total: 15.8s	remaining: 6.47s
80:	learn: 0.0018769	total: 18.1s	remaining: 4.23s
90:	learn: 0.0017851	total: 20.2s	remaining: 2s
99:	learn: 0.0017523	total: 22.2s	remaining: 0us


In [ ]:
cb_model.save_model('catboost_model.cbm')
candidates = pl.read_parquet('data/candidates_target.parquet')

# Делаем предсказания


In [10]:
events_test_2 = pl.read_parquet(r'data\events_test_2.parquet')

In [12]:
#events_test_2 = pl.from_pandas(events_test_2)
# Add the target column to events_labels with a value of 1
events_test_2 = events_test_2.with_columns(pl.lit(1).alias("target"))

# Perform an outer join on user_id and track_id
candidates2 = (
    events_test_2.select(["user_id", "track_id", "target"])
    .join(candidates, 
    on=["user_id", "track_id"], 
    how="outer")
)

# Fill null values in the target column with 0 and convert to integer
candidates2 = candidates2.with_columns(
    pl.col("target").fill_null(0).cast(pl.Int32)
)

# Sample 50 rows from the candidates DataFrame
print(candidates2.sample(50))

C:\Users\Admin\AppData\Local\Temp\ipykernel_13184\552681065.py:8: DeprecationWarning: Use of `how='outer'` should be replaced with `how='full'`.
  .join(candidates,


shape: (50, 9)
┌─────────┬──────────┬────────┬─────────────┬───┬───────────┬────────────┬────────────┬────────────┐
│ user_id ┆ track_id ┆ target ┆ user_id_rig ┆ … ┆ als_score ┆ user_genre ┆ user_artis ┆ target_rig │
│ ---     ┆ ---      ┆ ---    ┆ ht          ┆   ┆ ---       ┆ _affinity  ┆ t_ratio    ┆ ht         │
│ i32     ┆ i32      ┆ i32    ┆ ---         ┆   ┆ f64       ┆ ---        ┆ ---        ┆ ---        │
│         ┆          ┆        ┆ i64         ┆   ┆           ┆ f64        ┆ f64        ┆ i32        │
╞═════════╪══════════╪════════╪═════════════╪═══╪═══════════╪════════════╪════════════╪════════════╡
│ null    ┆ null     ┆ 0      ┆ 416929      ┆ … ┆ null      ┆ 0.092385   ┆ 0.003745   ┆ 0          │
│ null    ┆ null     ┆ 0      ┆ 1371188     ┆ … ┆ null      ┆ null       ┆ null       ┆ 1          │
│ null    ┆ null     ┆ 0      ┆ 517418      ┆ … ┆ 0.062938  ┆ null       ┆ null       ┆ 0          │
│ null    ┆ null     ┆ 0      ┆ 956702      ┆ … ┆ 0.17004   ┆ null       ┆ n

In [13]:
# Заполняем пропуски в столбце 'user_id' соответствующими значениями из 'user_id_right'
candidates2 = candidates2.with_columns(pl.col('user_id').fill_null(pl.col('user_id_right')).alias('user_id'))

# Заполняем пропуски в столбце 'track_id' соответствующими значениями из 'track_id_right'
candidates2 = candidates2.with_columns(pl.col('track_id').fill_null(pl.col('track_id_right')).alias('track_id'))

candidates2 = candidates2.select(['user_id',	'track_id',	'als_score',	'user_genre_affinity',	'user_artist_ratio','target'])
candidates2.write_parquet('data/candidates_to_rank.parquet')

In [6]:
candidates2 = pl.read_parquet('data/candidates_to_rank.parquet')

In [7]:
features = ['als_score', 'user_genre_affinity', 'user_artist_ratio']
target = 'target'
inference_data = Pool(data=np.array(candidates2[features]))
predictions = cb_model.predict_proba(inference_data)

candidates2 = candidates2.with_columns(pl.Series(predictions[:, 1]).alias('cb_score'))
candidates2

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score
i64,i64,f64,f64,f64,i32,f64
0,99262,null,0.076923,0.038462,0,0.000027
0,589498,null,0.192308,0.153846,0,0.000028
0,590262,null,0.192308,0.153846,0,0.000028
0,590303,null,0.192308,0.153846,0,0.000028
0,590692,null,0.192308,0.153846,0,0.000028
…,…,…,…,…,…,…
527,18862,null,null,null,1,0.999956
525255,85652278,null,null,null,1,0.999956
32303,37079539,null,null,null,1,0.999956


In [8]:
del predictions, inference_data
# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates2.sort(by=["user_id", "cb_score"], descending=[False, True])
del candidates2
candidates_to_rank

user_id,track_id,als_score,user_genre_affinity,user_artist_ratio,target,cb_score
i64,i64,f64,f64,f64,i32,f64
0,10327926,null,0.038462,0.115385,0,0.000028
0,17802671,null,0.115385,0.115385,0,0.000028
0,18102829,null,0.076923,0.115385,0,0.000028
0,589498,null,0.192308,0.153846,0,0.000028
0,590262,null,0.192308,0.153846,0,0.000028
…,…,…,…,…,…,…
1374582,76267532,null,0.285024,0.028986,0,0.000027
1374582,54559796,null,0.285024,0.028986,0,0.000027
1374582,54559802,null,0.285024,0.028986,0,0.000027


In [16]:
candidates_to_rank.write_parquet('data/cb_predictions.parquet')

In [15]:
candidates_to_rank = pl.read_parquet('data/cb_predictions.parquet')

# Оценка качества

In [17]:
def evaluate_recommendations(recommendations_df: pl.DataFrame, validation_df: pl.DataFrame, k: int):
   """
   Оценивает качество рекомендаций с прогресс-барами.

   Аргументы:
   recommendations_df (pl.DataFrame): DataFrame с рекомендациями.
   validation_df (pl.DataFrame): DataFrame с валидационными данными.
   k (int): Количество рекомендаций для оценки.

   Возвращает:
   dict: Словарь с метриками оценки рекомендаций.
   """
   # Инициализация переменных для метрик
   total_relevant = 0
   total_recommended = 0
   total_relevant_and_recommended = 0
   all_items = set()
   recommended_items = set()
   item_popularity = defaultdict(int)

   # Прогресс-бар для подсчёта популярности
   print("[1/4] Подсчёт популярности треков:")
   for track_id in tqdm(validation_df['track_id'], desc="Обработка треков"):
       item_popularity[track_id] += 1

   # Группировка с сортировкой по score
   print("\n[2/4] Группировка данных:")
   with tqdm(total=3, desc="Подготовка данных") as pbar:
       recommendations_grouped_df = recommendations_df.sort('score', descending=True).group_by('user_id').agg([
           pl.col('track_id').head(k).alias('track_id_list')
       ])
       pbar.update(1)
       
       validation_grouped_df = validation_df.group_by('user_id').agg([
           pl.col('track_id').implode().alias('track_id_list')
       ])
       pbar.update(1)
       
       recommendations_grouped = recommendations_grouped_df.to_dict(as_series=False)
       validation_grouped = validation_grouped_df.to_dict(as_series=False)
       validation_user_ids = set(validation_grouped['user_id'])
       pbar.update(1)

   # Обработка пользователей с прогресс-баром
   print("\n[3/4] Оценка рекомендаций:")
   user_ids = recommendations_grouped['user_id']
   total_users = len(user_ids)
   
   for idx, (user_id, recommended_tracks) in tqdm(
       enumerate(zip(user_ids, recommendations_grouped['track_id_list'])),
       total=total_users,
       desc="Обработка пользователей"
   ):
       if user_id not in validation_user_ids:
           continue
           
       real_tracks = validation_grouped['track_id_list'][validation_grouped['user_id'].index(user_id)]
       real_tracks_set = set(real_tracks)
       
       relevant_count = len(real_tracks_set)
       recommended_count = len(recommended_tracks)
       relevant_and_recommended = len(real_tracks_set & set(recommended_tracks))
       
       total_relevant += relevant_count
       total_recommended += recommended_count
       total_relevant_and_recommended += relevant_and_recommended
       
       for track_id in recommended_tracks:
           recommended_items.add(track_id)
           all_items.add(track_id)
           item_popularity[track_id] += 0

   # Расчёт novelty с прогресс-баром
   print("\n[4/4] Расчёт Novelty:")
   novelty_scores = []
   user_ids = recommendations_grouped['user_id']
   
   for user_id, recommended_tracks in tqdm(
       zip(user_ids, recommendations_grouped['track_id_list']),
       total=len(user_ids),
       desc="Обработка Novelty"
   ):
       if user_id not in validation_user_ids:
           continue
           
       for track_id in recommended_tracks:
           popularity = item_popularity.get(track_id, 1)
           try:
               novelty_scores.append(1.0 / popularity)
           except ZeroDivisionError:
               novelty_scores.append(0)

   # Вычисление финальных метрик
   metrics = {
       'recall': total_relevant_and_recommended / total_relevant if total_relevant > 0 else 0,
       'precision': total_relevant_and_recommended / total_recommended if total_recommended > 0 else 0,
       'coverage': len(recommended_items) / len(all_items) if len(all_items) > 0 else 0,
       'novelty': np.mean(novelty_scores) if novelty_scores else 0
   }
   
   print("\n✅ Оценка завершена")
   return metrics


In [19]:
k = 5  # Оцениваем топ-5 рекомендации
recommendations = candidates_to_rank.select(['user_id', 'track_id','cb_score']).rename({
    'cb_score': 'score'
})

#validation = pl.from_pandas(events_test_2[['user_id', 'track_id']])
validation = events_test_2.select(['user_id', 'track_id'])
# Вычисление метрик
metrics1 = evaluate_recommendations(recommendations, validation, k)
print(metrics1)

[1/4] Подсчёт популярности треков:


Обработка треков: 100%|██████████| 7522628/7522628 [00:02<00:00, 3235157.34it/s]



[2/4] Группировка данных:


Подготовка данных: 100%|██████████| 3/3 [00:25<00:00,  8.38s/it]



[3/4] Оценка рекомендаций:


Обработка пользователей: 100%|██████████| 1373221/1373221 [21:28<00:00, 1065.69it/s]



[4/4] Расчёт Novelty:


Обработка Novelty: 100%|██████████| 1373221/1373221 [00:01<00:00, 1227184.10it/s]



✅ Оценка завершена
{'recall': 0.1924019106088989, 'precision': 0.4542220138416188, 'coverage': 1.0, 'novelty': np.float64(0.05255849802319558)}


In [20]:
pops = pl.read_parquet('data/top_popular.parquet')
pops = pops.with_columns(pl.lit(1).alias("score"))
pops =pops.select(['user_id', 'recommended_track_id','score']).rename({
    'recommended_track_id': 'track_id'
})
metrics2 = evaluate_recommendations(pops, validation, k)
print(metrics2)

[1/4] Подсчёт популярности треков:


Обработка треков: 100%|██████████| 7522628/7522628 [00:02<00:00, 3411364.96it/s]



[2/4] Группировка данных:


Подготовка данных: 100%|██████████| 3/3 [00:00<00:00,  3.71it/s]



[3/4] Оценка рекомендаций:


Обработка пользователей: 100%|██████████| 783525/783525 [21:28<00:00, 608.08it/s] 



[4/4] Расчёт Novelty:


Обработка Novelty: 100%|██████████| 783525/783525 [00:00<00:00, 1239237.76it/s]



✅ Оценка завершена
{'recall': 0.0018010992966819574, 'precision': 0.004251593680818249, 'coverage': 1.0, 'novelty': np.float64(0.005224810340302745)}


In [27]:
als_recs = pl.read_parquet('data/als_scores.parquet')
als_recs

score,user_id,track_id
f64,i64,i64
0.295663,3,45499814
0.27522,3,33311009
0.246964,3,54798445
0.241626,3,39946957
0.210551,3,57921154
…,…,…
0.084045,1374582,48591706
0.084006,1374582,47201922
0.083638,1374582,76030982


In [28]:
metrics3 = evaluate_recommendations(pops, validation, k)
print(metrics3)              

[1/4] Подсчёт популярности треков:


Обработка треков: 100%|██████████| 7522628/7522628 [00:02<00:00, 3335977.12it/s]



[2/4] Группировка данных:


Подготовка данных: 100%|██████████| 3/3 [00:00<00:00,  3.78it/s]



[3/4] Оценка рекомендаций:


Обработка пользователей: 100%|██████████| 783525/783525 [25:32<00:00, 511.13it/s] 



[4/4] Расчёт Novelty:


Обработка Novelty: 100%|██████████| 783525/783525 [00:00<00:00, 1141435.80it/s]



✅ Оценка завершена
{'recall': 0.0018010992966819574, 'precision': 0.004251593680818249, 'coverage': 1.0, 'novelty': np.float64(0.0052248103403027425)}


Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

In [29]:
# Создаем DataFrame из словарей
eva = pd.DataFrame([metrics1, metrics2, metrics3], index=['Cb-model', 'Most popular', 'ALS-personalized'])

display(eva)

,recall,precision,coverage,novelty
Cb-model,0.192402,0.454222,1.0,0.052558
Most popular,0.001801,0.004252,1.0,0.005225
ALS-personalized,0.001801,0.004252,1.0,0.005225


# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.

Комплексная характеристика качества рекомендательной системы на основе предоставленных метрик:
1. Recall (Полнота): 0.192 (19.2%)
Интерпретация: Система "вспоминает" менее 20% релевантных (актуальных) элементов для пользователя.
Анализ:
Низкий recall указывает на то, что система пропускает много релевантных рекомендаций. Возможные причины:
Слишком строгие критерии отбора (например, высокий порог для попадания в топ-k).
Ограниченная способность модели улавливать разнообразные интересы пользователей.
Недостаточная персонализация (например, перекос в сторону популярных или "безопасных" рекомендаций).

2. Precision (Точность): 0.454 (45.4%)
Интерпретация: Почти половина рекомендаций в топе являются релевантными.
Анализ:
Высокая точность говорит о том, что система хорошо фильтрует шум и делает осмысленные предсказания.
Риск: Высокая точность при низком recall может быть признаком "переобучения" на узкий набор паттернов.
3. Coverage (Покрытие): 1.0 (100%)
Интерпретация: Система может рекомендовать 100% элементов каталога.
Анализ:
Идеальное покрытие — редкий случай. Это означает:
Отсутствие "холодных стартов" для новых товаров/контента.
Возможность рекомендовать нишевые элементы.
Потенциальные проблемы:
Высокое покрытие может достигаться за счет рекомендаций "всего подряд" (но это противоречит хорошей precision).
4. Novelty (Новизна): 0.0525
Интерпретация: Рекомендации имеют крайне низкую новизну (близко к нулю).
Анализ:
Система рекомендует преимущественно популярные/известные элементы.
Причины:
Доминирование popularity-based алгоритмов в финальном ранжировании.
Риски:
Пользователи видят только популярный контент.
Снижение долгосрочной вовлеченности из-за предсказуемости.


**Сводный анализ:**  
*Сильные стороны:*  
Высокая точность (precision) при полном покрытии каталога.
Стабильность работы (низкий риск ошибок из-за 100% coverage).
*Слабые стороны:*  
Низкая полнота (recall) — система "не видит" много релевантных вариантов.
Крайне низкая новизна — рекомендации шаблонны и неперсонализированы.
*Баланс метрик:*  
Precision-Recall Tradeoff: Система жертвует recall ради precision, что типично для алгоритмов с жестким порогом отсечения.  
Novelty-Coverage Paradox: Полное покрытие достигнуто за счет рекомендаций популярного контента, что снижает новизну.  
  
**Индустриальный контекст:**  
Для рекомендательных систем в e-commerce типичный precision варьируется между 20-40%, что делает текущий результат (45.4%) выше среднего.
Низкая новизна (0.0525) может быть критичной для платформ, где discovery контента — ключевая ценность (например, музыкальные сервисы).
Coverage=1.0 — вообще на какой-то артефакт похоже.